<a href="https://colab.research.google.com/github/richmondvan/melanoma-detection/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training

Import all modules and mount Google Drive

In [3]:
# Must be run every time!

import pathlib # Manage file paths
import math # Manage basic math
import pickle # Storing epoch number
import csv # Storing data in .csv files
from google.colab import drive # For mounting GDrive
import tensorflow_hub as hub # For importing EfficientNet

import tensorflow as tf #nightly
from tensorflow.keras import models, layers, losses, metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import CSVLogger

# Mount Google Drive
drive.mount('/content/gdrive') 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Prepare datasets

In [4]:
# Setting up file paths
PATH = "/content/gdrive/My Drive/Dataset/DatasetSorted/"

TRAINING_PATH = pathlib.Path(PATH + "training/")
VALIDATION_PATH = pathlib.Path(PATH + "validation/")
TEST_PATH = pathlib.Path(PATH + "test/")

# Create image generators
train_image_generator = ImageDataGenerator(
    rescale=1./255,
    brightness_range=(0.95, 1.05),
    horizontal_flip=True,
    vertical_flip=True) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True) # Generator for our validation data

# Some constants
batch_size = 32
epochs = 15
IMG_HEIGHT = 600
IMG_WIDTH = 600
TRAIN_LEN = len(list(TRAINING_PATH.glob("*/*.jpg")))
VALID_LEN = len(list(VALIDATION_PATH.glob("*/*.jpg")))
CLASS_NAMES = ['benign', 'malignant']

# Get generated datasets
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=TRAINING_PATH,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary',
                                                           classes=CLASS_NAMES)

val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=VALIDATION_PATH,
                                                              shuffle=True,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary',
                                                              classes=CLASS_NAMES)

Found 8423 images belonging to 2 classes.
Found 1054 images belonging to 2 classes.


Prepare metrics and weights

In [0]:
# Get some training weights to offset class imbalance
numBenign = len(list(TRAINING_PATH.glob("benign/*.jpg")))
numMalignant = len(list(TRAINING_PATH.glob("malignant/*.jpg")))
total = numBenign + numMalignant
weight_for_0 = (1 / numBenign)*(total)/2.0 
weight_for_1 = (1.5 / numMalignant)*(total)/2.0
class_weight = {0: weight_for_0, 1: weight_for_1}

# Metrics we will be using to assess accuracy
METRICS = [
      metrics.TruePositives(name='tp'),
      metrics.FalsePositives(name='fp'),
      metrics.TrueNegatives(name='tn'),
      metrics.FalseNegatives(name='fn'), 
      metrics.BinaryAccuracy(name='accuracy'),
      metrics.Precision(name='precision'),
      metrics.Recall(name='recall'),
      metrics.AUC(name='auc'),
]

Prepare model

In [6]:
# Hyperparameters
NEURONS_PER_LAYER = 1024
REG_LAMBDA = 0.001
DROPOUT = 0.1
ACTIVATION = "relu"

# Build model
model = models.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/efficientnet/b7/feature-vector/1", trainable=False),
    layers.Dropout(DROPOUT),
    layers.Dense(NEURONS_PER_LAYER, kernel_regularizer=tf.keras.regularizers.l2(REG_LAMBDA), activation=ACTIVATION),
    layers.Dropout(DROPOUT),
    layers.Dense(NEURONS_PER_LAYER, kernel_regularizer=tf.keras.regularizers.l2(REG_LAMBDA), activation=ACTIVATION),
    layers.Dropout(DROPOUT),
    layers.Dense(NEURONS_PER_LAYER, kernel_regularizer=tf.keras.regularizers.l2(REG_LAMBDA), activation=ACTIVATION),
    layers.Dropout(DROPOUT),
    layers.Dense(NEURONS_PER_LAYER, kernel_regularizer=tf.keras.regularizers.l2(REG_LAMBDA), activation=ACTIVATION),
    layers.Dropout(DROPOUT),
    layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=METRICS)

model.build([None, IMG_HEIGHT, IMG_WIDTH, 3])
model.summary()



Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     multiple                  64097680  
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  2622464   
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  1049600   
_________________________________________________________________
dropout_2 (Dropout)          multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple                  1

Load model weights and last epoch

In [13]:
# Get last epoch number from pickled file

EPOCH_FILEPATH = f"/content/gdrive/My Drive/Dataset/enet{NEURONS_PER_LAYER}_4_epochnum"

infile = open(EPOCH_FILEPATH, 'rb')
try: 
    infile.seek(0)
    epoch = pickle.load(infile)
    model.load_weights(f"/content/gdrive/My Drive/Dataset/enet{NEURONS_PER_LAYER}_4/epoch{epoch}.h5")
except: 
    # Otherwise start again (only happens if no epoch number found)
    epoch = 0


print(epoch)
infile.close()

/content/gdrive/My Drive/Dataset/enet1024_4_epochnum
16


Prepare CSV logger

In [0]:
# File where we store our CSV history

HISTORY_FILE = f'/content/gdrive/My Drive/Dataset/enet{NEURONS_PER_LAYER}_stats.csv'

csv_logger = CSVLogger(HISTORY_FILE, append=True)

Train model

In [0]:
# Train for 150 epochs

if epoch < 150:
    for i in range(epoch, 150):
        history = model.fit(x=train_data_gen, 
                            epochs=i+1, 
                            initial_epoch=i, 
                            verbose=1, 
                            validation_data=val_data_gen, 
                            validation_steps=VALID_LEN // batch_size, 
                            steps_per_epoch=TRAIN_LEN // batch_size, 
                            class_weight=class_weight,
                            callbacks = [csv_logger])
        model.save_weights(f"/content/gdrive/My Drive/Dataset/enet{NEURONS_PER_LAYER}_4/epoch{i + 1}.h5")
        outfile = open(EPOCH_FILEPATH, 'wb')
        pickle.dump(i+1, outfile)
        outfile.close()

    
    epoch = 150

Epoch 17/17
 42/263 [===>..........................] - ETA: 36:57 - loss: 0.7227 - tp: 132.0000 - fp: 211.0000 - tn: 942.0000 - fn: 59.0000 - accuracy: 0.7991 - precision: 0.3848 - recall: 0.6911 - auc: 0.8134

Fine-tuning with trainable EfficientNet

In [0]:
# Build model again

model = models.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/efficientnet/b7/feature-vector/1", trainable=True), # Trainable this time
    layers.Dropout(DROPOUT),
    layers.Dense(NEURONS_PER_LAYER, kernel_regularizer=tf.keras.regularizers.l2(REG_LAMBDA), activation=ACTIVATION),
    layers.Dropout(DROPOUT),
    layers.Dense(NEURONS_PER_LAYER, kernel_regularizer=tf.keras.regularizers.l2(REG_LAMBDA), activation=ACTIVATION),
    layers.Dropout(DROPOUT),
    layers.Dense(NEURONS_PER_LAYER, kernel_regularizer=tf.keras.regularizers.l2(REG_LAMBDA), activation=ACTIVATION),
    layers.Dropout(DROPOUT),
    layers.Dense(NEURONS_PER_LAYER, kernel_regularizer=tf.keras.regularizers.l2(REG_LAMBDA), activation=ACTIVATION),
    layers.Dropout(DROPOUT),
    layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), # smaller learning rate
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=METRICS)

model.build([None, IMG_HEIGHT, IMG_WIDTH, 3])
model.summary()

model.load_weights(f"/content/gdrive/My Drive/Dataset/enet{NEURONS_PER_LAYER}_4/epoch{epoch}.h5")

Train again

In [0]:
# Train for 50 more epochs

if epoch < 200:
    for i in range(epoch, 200):
        history = model.fit(x=train_data_gen, 
                            epochs=i+1, 
                            initial_epoch=i, 
                            verbose=1, 
                            validation_data=val_data_gen, 
                            validation_steps=VALID_LEN // batch_size, 
                            steps_per_epoch=TRAIN_LEN // batch_size, 
                            class_weight=class_weight,
                            callbacks = [csv_logger])
        model.save_weights(f"/content/gdrive/My Drive/Dataset/enet{NEURONS_PER_LAYER}_4/epoch{i + 1}.h5")
        outfile = open(EPOCH_FILEPATH, 'wb')
        pickle.dump(i+1, outfile)
        outfile.close()